In [2]:
import os
from os import listdir,makedirs
from os.path import isfile,join
import numpy as np
from PIL import Image
from matplotlib import pyplot as plt
import tensorflow as tf
import keras.backend as K
import keras
from keras.models import Sequential
import cv2



C:\Users\Meers\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [26]:
'''
Hyper-Parameters
'''
image_H = 96.0
image_W = 96.0
grayscale = True
batch_size = 16
learning_rate=0.001
momentum = 0
decay = 0
epochs = 100
droupout= 0.1
gradient_clipping = 0.1
nFilters = 128
nlandmarks = nLabels = 68
K=45
img_rows = 96
img_cols = 96
img_channels = 1

In [98]:
# Get files
import pts_loader
img_x = []
pts_y = []
for i in range(1,50):
    if i < 10:
        img_f = r"C:\Users\Meers\Documents\Techie\InkersIntern\Phase2\FLDCode\300W\300w\01_Indoor\indoor_00" + str(i) +".png"
        pts_f = r"C:\Users\Meers\Documents\Techie\InkersIntern\Phase2\FLDCode\300W\300w\01_Indoor\indoor_00" + str(i) +".pts"
    elif i < 100:
        img_f = r"C:\Users\Meers\Documents\Techie\InkersIntern\Phase2\FLDCode\300W\300w\01_Indoor\indoor_0" + str(i) +".png"
        pts_f = r"C:\Users\Meers\Documents\Techie\InkersIntern\Phase2\FLDCode\300W\300w\01_Indoor\indoor_0" + str(i) +".pts"
    else:
        img_f = r"C:\Users\Meers\Documents\Techie\InkersIntern\Phase2\FLDCode\300W\300w\01_Indoor\indoor_" + str(i) +".png"
        pts_f = r"C:\Users\Meers\Documents\Techie\InkersIntern\Phase2\FLDCode\300W\300w\01_Indoor\indoor_" + str(i) +".pts"
    pts = pts_loader.load(pts_f)
    # Loading imgae in grayscale
    img = cv2.imread(img_f)
    
    scale_x = image_W/img.shape[0]
    scale_y = image_H/img.shape[1]
    img = cv2.resize(img, (img_rows, img_cols))
    img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    img = img.reshape(img_rows, img_cols, 1)
    landmarks = []
    n = 0
    for p in pts:
        new_pts = []
        new_pts = (p[0] * scale_y , p[1] * scale_x )
        #new_points.append(new_pts)
        x = int(new_pts[0])
        y = int(new_pts[1])
        #print(n, x, y)
        landmarks.append(np.zeros((96, 96, 1), np.uint8))
        landmarks[n][x][y][0] = 255 #WHite
        n = n +1
    img_x.append(img)
    pts_y.append(landmarks)


0 42 11
1 42 15
2 43 19
3 44 22
4 45 25
5 46 28
6 48 31
7 50 33
8 52 34
9 55 33
10 57 31
11 58 28
12 59 25
13 60 22
14 61 19
15 61 15
16 61 11
17 44 15
18 46 13
19 48 14
20 49 15
21 51 16
22 54 15
23 56 14
24 57 13
25 59 13
26 60 14
27 53 18
28 53 20
29 53 22
30 53 24
31 50 25
32 51 25
33 53 26
34 54 25
35 55 25
36 46 17
37 47 16
38 49 17
39 50 18
40 49 18
41 47 18
42 55 17
43 56 16
44 57 16
45 59 16
46 58 17
47 56 18
48 49 29
49 51 28
50 52 28
51 53 28
52 54 28
53 55 28
54 56 28
55 55 30
56 54 32
57 53 32
58 52 32
59 51 31
60 50 29
61 52 29
62 53 29
63 54 29
64 55 28
65 54 30
66 53 30
67 52 30
0 23 38
1 24 47
2 26 57
3 27 65
4 30 73
5 33 79
6 37 84
7 41 87
8 46 87
9 50 85
10 54 80
11 57 74
12 59 66
13 61 58
14 61 50
15 61 42
16 62 33
17 27 41
18 29 38
19 33 38
20 36 41
21 40 42
22 46 41
23 48 39
24 52 35
25 55 33
26 58 35
27 42 44
28 42 49
29 43 54
30 43 60
31 40 62
32 41 63
33 43 63
34 45 62
35 47 61
36 30 43
37 32 42
38 35 42
39 38 45
40 35 45
41 32 45
42 47 43
43 50 40
44 52 39
45 

2 42 43
3 44 46
4 45 48
5 48 50
6 50 52
7 53 53
8 57 53
9 60 53
10 64 51
11 68 49
12 70 47
13 72 44
14 73 40
15 72 37
16 72 33
17 39 34
18 40 33
19 43 32
20 46 33
21 48 34
22 52 33
23 54 31
24 58 30
25 62 30
26 65 30
27 51 35
28 51 37
29 51 39
30 51 41
31 49 43
32 51 43
33 52 43
34 54 42
35 56 42
36 43 37
37 44 36
38 46 36
39 48 37
40 47 37
41 44 38
42 56 35
43 58 34
44 60 33
45 62 34
46 61 35
47 58 35
48 49 47
49 50 46
50 51 45
51 53 45
52 55 44
53 58 45
54 61 45
55 59 47
56 56 48
57 55 48
58 53 48
59 51 48
60 49 47
61 52 46
62 54 46
63 56 46
64 61 45
65 56 46
66 54 47
67 52 47
0 24 42
1 25 49
2 26 56
3 28 63
4 31 69
5 35 74
6 39 78
7 44 81
8 49 82
9 55 81
10 59 79
11 62 75
12 66 71
13 68 65
14 69 60
15 69 53
16 69 48
17 29 45
18 33 43
19 36 42
20 40 42
21 44 43
22 50 43
23 53 41
24 57 40
25 62 39
26 64 41
27 46 46
28 46 51
29 46 55
30 46 59
31 41 61
32 44 62
33 47 63
34 49 62
35 52 61
36 32 47
37 35 46
38 37 46
39 40 47
40 38 47
41 35 48
42 50 46
43 53 44
44 56 44
45 58 45
46 56 46
4

51 70 51
52 70 50
53 71 51
54 72 54
55 71 55
56 71 56
57 70 56
58 68 56
59 67 55
60 66 54
61 69 52
62 70 52
63 70 52
64 72 53
65 70 53
66 70 53
67 69 53
0 14 28
1 15 34
2 17 38
3 19 42
4 21 46
5 25 49
6 29 51
7 33 53
8 38 54
9 43 53
10 48 52
11 52 50
12 56 47
13 58 42
14 59 38
15 60 33
16 61 28
17 17 23
18 20 22
19 25 22
20 29 22
21 33 23
22 41 23
23 44 21
24 50 21
25 54 21
26 58 23
27 38 27
28 38 30
29 38 32
30 38 35
31 34 38
32 36 39
33 38 39
34 41 38
35 43 38
36 22 28
37 25 26
38 28 26
39 32 28
40 29 28
41 25 28
42 44 27
43 48 25
44 50 25
45 53 26
46 51 27
47 48 27
48 30 45
49 33 44
50 36 42
51 39 42
52 41 41
53 46 42
54 49 44
55 46 46
56 42 48
57 40 48
58 37 48
59 33 47
60 31 45
61 36 43
62 39 43
63 41 43
64 48 44
65 42 45
66 39 45
67 37 45
0 54 44
1 55 49
2 56 55
3 58 59
4 60 64
5 62 68
6 64 72
7 67 75
8 71 75
9 75 74
10 78 71
11 81 66
12 82 61
13 84 55
14 84 49
15 85 44
16 86 36
17 54 42
18 55 38
19 58 37
20 61 38
21 64 39
22 68 38
23 70 35
24 74 35
25 77 35
26 80 37
27 66 42
28 

In [99]:
print(img_x[0].shape)

train_x = []
train_y = []
test_x = []
test_y = []


(96, 96, 1)


In [125]:
train_x = np.array(img_x[:24])
train_y = np.array(pts_y[:24])
test_x = np.array(img_x[24:])
test_y = np.array(pts_y[24:])


In [126]:
print(train_y[0].shape)

(68, 96, 96, 1)


In [127]:
f = open(r'C:\Users\Meers\Documents\Techie\InkersIntern\Phase2\Merget\Merget\network\config\kernel45.txt')
kernel45 = np.fromfile(f, dtype = np.float32, count = -1, sep = " ")

In [128]:
from keras.layers import Dense, Dropout, Activation, Add
from keras.layers import Convolution2D, Lambda, Input, Reshape
from keras.layers.normalization import BatchNormalization
from keras.models import Model

def apply_kernel(local_out):
    '''
     get kernel from text file
     tile it to 1x1x45x45
     convolve with local out
    '''
    f = open(r'C:\Users\Meers\Documents\Techie\InkersIntern\Phase2\Merget\Merget\network\config\kernel45.txt')
    kernel45 = np.fromfile(f, dtype = np.float32, count = -1, sep = " ")
    
    # [1 x 1 x 45 x 45]
    kernel45 = np.reshape(kernel45, (45, 45, 1, 1))
    # [numLabels x 1 x 45 x 45] if tensor convolution is used
    kernel_stacked 	= np.tile(kernel45, (1, 1, nLabels,1))
    #heatmaps = Convolution2D(local_out, kernel_stacked)
    kernel_tensor = tf.convert_to_tensor(kernel_stacked, dtype=tf.float32)
    print("local_out", local_out)
    print("kernel", kernel_tensor)
    '''
    May be use this for group Convolution
    import tensorlayer as tl
    tl.layers.GroupConv2d(prev_layer, n_filter=32, filter_size=(3, 3), strides=(2, 2), n_group=2, act=None,
    padding='SAME', W_init=<tensorflow.python.ops.init_ops.TruncatedNormal object>, 
    b_init=<tensorflow.python.ops.init_ops.Constant object>, W_init_args=None, b_init_args=None, name='groupconv')
    '''
    return tf.nn.conv2d(local_out, kernel_tensor, strides=[1,1,1,1], padding="SAME")
    #return heatmaps


'''
LOCAL SUBNET
'''

# [ImageC x ImageH x ImageW]

input_img = Input(( img_rows, img_cols, img_channels))
model = Convolution2D(filters=nFilters, kernel_size=(5, 5), padding='same', activation='relu')(input_img)
model = BatchNormalization()(model)
model = Convolution2D(filters=nFilters, kernel_size=(5, 5), padding='same', activation='relu')(model)
model = BatchNormalization()(model)
model = Convolution2D(filters=nFilters, kernel_size=(5, 5), padding='same', activation='relu')(model)
model = BatchNormalization()(model)
model = Convolution2D(filters=nFilters, kernel_size=(5, 5), padding='same', activation='relu')(model)
model = BatchNormalization()(model)
model = Convolution2D(filters=nFilters, kernel_size=(5, 5), padding='same', activation='relu')(model)
model = BatchNormalization()(model)

model = Convolution2D(filters=nFilters, kernel_size=(3, 3), padding='same', activation='relu')(model)
model = BatchNormalization()(model)
model = Convolution2D(filters=nFilters, kernel_size=(3, 3), padding='same', activation='relu')(model)
model = BatchNormalization()(model)
model = Convolution2D(filters=nFilters, kernel_size=(3, 3), padding='same', activation='relu')(model)
model = BatchNormalization()(model)
model = Convolution2D(filters=nFilters, kernel_size=(3, 3), padding='same', activation='relu')(model)
model = BatchNormalization()(model)
model = Convolution2D(filters=nFilters, kernel_size=(3, 3), padding='same', activation='relu')(model)
model = BatchNormalization()(model)
model = Convolution2D(filters=nFilters, kernel_size=(3, 3), padding='same', activation='relu')(model)
model = BatchNormalization()(model)
model = Convolution2D(filters=nFilters, kernel_size=(3, 3), padding='same', activation='relu')(model)
model = BatchNormalization()(model)
model = Convolution2D(filters=nFilters, kernel_size=(3, 3), padding='same', activation='relu')(model)
model = BatchNormalization()(model)
model = Convolution2D(filters=nFilters, kernel_size=(3, 3), padding='same', activation='relu')(model)
model = BatchNormalization()(model)
model = Convolution2D(filters=nFilters, kernel_size=(3, 3), padding='same', activation='relu')(model)
model = BatchNormalization()(model)
print("Local subnet", model)
'''
1x1 Convolution with activation
'''
model_Conv = Convolution2D(filters=nLabels, kernel_size=(1, 1),activation='relu')(model)
model_Conv = Lambda(apply_kernel, output_shape=(nLabels, img_cols, img_rows))(model_Conv)
print("Main 1x1", model_Conv)
'''
Ol 1x1 Linear convolution
Apply Gaussian Kernel filter
'''
model_Ol = Convolution2D(filters=nLabels, kernel_size=(1, 1),activation=None)(model)
# What about tiling the kernels????
model_Ol = Lambda(apply_kernel, output_shape=(nLabels, img_cols, img_rows), name='Ol')(model_Ol)
print("Local output", model_Ol)

output_Ol = keras.layers.concatenate([model_Conv, model_Ol])
print("Concatenate", output_Ol)
output_Ol = Reshape((96, 96,-1))(output_Ol)
print("Reshape", output_Ol)

'''
GLOBAL SUBNET
'''
model = Convolution2D(filters=nFilters, kernel_size=(3, 3), padding='same', dilation_rate = 4, activation='relu')(model)
model = BatchNormalization()(model)
model = Convolution2D(filters=nFilters, kernel_size=(3, 3), padding='same', dilation_rate = 4, activation='relu')(model)
model = BatchNormalization()(model)
model = Convolution2D(filters=nFilters, kernel_size=(3, 3), padding='same', dilation_rate = 4, activation='relu')(model)
model = BatchNormalization()(model)
model = Convolution2D(filters=nFilters, kernel_size=(3, 3), padding='same', dilation_rate = 4, activation='relu')(model)
model = BatchNormalization()(model)
model = Convolution2D(filters=nFilters, kernel_size=(3, 3), padding='same', dilation_rate = 4, activation='relu')(model)
model = BatchNormalization()(model)
model = Convolution2D(filters=nFilters, kernel_size=(3, 3), padding='same', dilation_rate = 4, activation='relu')(model)
model = BatchNormalization()(model)
model = Convolution2D(filters=nFilters, kernel_size=(3, 3), padding='same', dilation_rate = 4, activation='relu')(model)
model = BatchNormalization()(model)
model = Convolution2D(filters=nLabels, kernel_size=(1, 1), activation=None)(model)
FLD_model = Model(inputs = input_img, outputs=model)
FLD_model.summary()


Local subnet Tensor("batch_normalization_279/cond/Merge:0", shape=(?, 96, 96, 128), dtype=float32)
local_out Tensor("conv2d_315/Relu:0", shape=(?, 96, 96, 68), dtype=float32)
kernel Tensor("lambda_13/Const:0", shape=(45, 45, 68, 1), dtype=float32)
Main 1x1 Tensor("lambda_13/Conv2D:0", shape=(?, 96, 96, 1), dtype=float32)
local_out Tensor("conv2d_316/BiasAdd:0", shape=(?, 96, 96, 68), dtype=float32)
kernel Tensor("Ol_12/Const:0", shape=(45, 45, 68, 1), dtype=float32)
Local output Tensor("Ol_12/Conv2D:0", shape=(?, 96, 96, 1), dtype=float32)
Concatenate Tensor("concatenate_13/concat:0", shape=(?, 96, 96, 2), dtype=float32)
Reshape Tensor("reshape_13/Reshape:0", shape=(?, 96, 96, 136), dtype=float32)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_13 (InputLayer)        (None, 96, 96, 1)         0         
_________________________________________________________________
conv2d_300 (Conv2D)          

* Loss Function: Weighted Square Error
* Exclude non-labeled landmarks from the loss
* Weight each of N landmarks depending on whether they lie within the image boundary:
\begin{equation*}
wij = V (i,j) + \frac{L(i,j)}{10} + \frac{L(i,j)}{2N}\sum_{k=1}^N V (i,k),       \\[1pt]
\text{    where V (i; j) and L(i; j) are indicator functions with L(i; j) = 1 iff landmark j in face i is labeled}    \\[1pt]
\text {V (i; j) = 1 iff landmark j in face i is within the image boundary, zero otherwise.    }\\[1pt]
\text{Note that L(i; j) = 0 implies V (i; j) = 0.}\end{equation*}

\begin{equation*}
e_i = \sum_{n=1}^N w_{in}(((O^l_n - G_n) * K)^2 + (O^g_n - G_n * K)^2), \\[2pt]
\text{    where    } \\[2pt]
O^l_n    and    O^g_n = \text{nth channels of the local- and global context subnet outputs, respectively.} \\[1pt]
\text{K = kernel }   \\[1pt]
\text{Gn = ground truth heatmap for landmark n }   \\[1pt]
\text{For efficiency, the kernel convolution after stacking is reused for the loss}
\end{equation*}


In [141]:
import tensorflow as tf
from keras import backend as K

def landmark_weight(labels):
    #[numLabels x 1 x 1]
    Win = tf.reduce_mean(labels)
    Win = tf.multiply(Win, 0.5)
    return Win

'''
Ol - [numLabels x ImageH x ImageW] Kernel conv already applied in Network
Og - [numLabels x ImageH x ImageW]
labels - Landmark coordinates [numLabels x ImageH x ImageW] without heatmaps
'''
def loss_function(Og, labels):#Ol, Og,labels):
    
    # Gn = labels x K
    #Gn = apply_kernel(labels)
    Gn = labels

    Win = landmark_weight(labels)
    
    # Og - Gn
    global_diff = tf.subtract(Og, Gn)
    global_err = tf.square(global_diff)
    
    # Ol - Gn
    '''
    Ol = FLD_model.get_layer(name='Ol')...
    How to access intermediate layer or model from here???
    Currently accessing since model from global...
    
    local_diff = tf.subtract(model_Ol, Gn)
    local_err = tf.square(local_diff)
    
    comb_err = tf.add(local_err, global_err)
    
    wt_square_err = tf.reduce_sum(Win * comb_err)
    return wt_square_err    
    '''
    return tf.square(global_err)


In [142]:
FLD_model.compile(optimizer='sgd',
              loss=loss_function,
              metrics=['accuracy'])


In [143]:
 FLD_model.fit(train_x, train_y, batch_size=8, epochs=1) #, validation_data=(test_x, test_y))

Epoch 1/1


ValueError: Cannot feed value of shape (8, 68, 96, 96, 1) for Tensor 'conv2d_324_target_4:0', which has shape '(?, ?, ?, ?)'

In [38]:
import cntk as C

cntk_model = C.load_model(r"C:\Users\Meers\Documents\Techie\InkersIntern\Phase2\Merget\Merget\python\output\models\model.21")

In [39]:
cntk_model.parameters

(Parameter('W', [], [68 x 128 x 1 x 1]),
 Parameter('b', [], [68 x 1 x 1]),
 Parameter('scale', [], [128]),
 Parameter('bias', [], [128]),
 Parameter('W', [], [128 x 128 x 3 x 3]),
 Parameter('b', [], [128 x 1 x 1]),
 Parameter('scale', [], [128]),
 Parameter('bias', [], [128]),
 Parameter('W', [], [128 x 128 x 3 x 3]),
 Parameter('b', [], [128 x 1 x 1]),
 Parameter('scale', [], [128]),
 Parameter('bias', [], [128]),
 Parameter('W', [], [128 x 128 x 3 x 3]),
 Parameter('b', [], [128 x 1 x 1]),
 Parameter('scale', [], [128]),
 Parameter('bias', [], [128]),
 Parameter('W', [], [128 x 128 x 3 x 3]),
 Parameter('b', [], [128 x 1 x 1]),
 Parameter('scale', [], [128]),
 Parameter('bias', [], [128]),
 Parameter('W', [], [128 x 128 x 3 x 3]),
 Parameter('b', [], [128 x 1 x 1]),
 Parameter('scale', [], [128]),
 Parameter('bias', [], [128]),
 Parameter('W', [], [128 x 128 x 3 x 3]),
 Parameter('b', [], [128 x 1 x 1]),
 Parameter('scale', [], [128]),
 Parameter('bias', [], [128]),
 Parameter('W',

In [42]:
cntk_model.save("cntk_model.h5")

In [47]:
FLD = keras.models.load_model(r"C:\Users\Meers\Documents\Techie\InkersIntern\Phase2\FLDCode\cntk_model.h5")

OSError: Unable to open file (File signature not found)

In [48]:
import keras.backend as K

#save_model_hdf5(cntk_model, 'my_model.h5')
kerasmodel = K.functional_ops. load_model(cntk_model)

AttributeError: module 'keras.backend' has no attribute 'load_model'

In [21]:
#func = cntk_model.outputs[0]
cntk_model.save('cntk_model.onnx', format=C.ModelFormat.)

In [24]:
import onnx
onnx_model = onnx.load('cntk_model.onnx')

In [26]:
from onnx_tf.backend import prepare
tf_model = prepare(onnx_model)

ValidationError: Required attribute 'consumed_inputs' is missing.

==> Context: Bad node spec: input: "Plus5153_Output_0" input: "Parameter106" input: "Parameter107" input: "Constant108" input: "Constant109" output: "BatchNormalization5165_Output_0" name: "BatchNormalization5165" op_type: "BatchNormalization" attribute { name: "spatial" i: 1 type: INT } attribute { name: "is_test" i: 1 type: INT } attribute { name: "epsilon" f: 1e-05 type: FLOAT } attribute { name: "momentum" f: 0 type: FLOAT } doc_string: "" domain: ""